In [1]:
#Bowerbird null model (all males are guarders)

#Key
#Notes- Paramters contain underscores, while functions don't
#SB: Stay at bower
#FG: Foraging
#MT: Maraud travel (travel to rival's bower)
#MA: Maraud action (destroy bower if absent, have antagonistic interaction in present)
#MR: Maraud return (return to own bower)
#RB: Repair own bower
#MT_vs_FG: the probability of next transition away from SB being to MT (otherwise it would be to FG)
#a=-1: denotes a staying at bower action on ticket
#a=-2: denotes a foraging action on ticket
#a=-3: denotes a female visiting action on a ticket
#a=-4: denotes travel to another male's bower
#a=-5: denotes marauding of another male's bower
#a=-6: denotes returning from another male's bower
#a=-7: denotes a bower repair action on a ticket
#targ: denotes travel, maurading action, and return from said target's bower
#networkwriter: generates network where everybird is connected
#addtotimeline: adds tickets to timeline
#travel_times_linear_p: generates a matrix that contains the probabilities of travelling to a given a bower 
#if the relationship between distance and travel preference is linear
#improb: The probability of traveling improb_distance or less
#improb_distance: The distance at which there is only a (1-improb)% chance of choosing to travel
#lamb: lambda calculated by solving improb=1-numpy.exp(-lamb*improb_distance)
#bower_states: options are 1(bower intact) and 0 (bower destroyed-will never be the case when all guarders)
#male_states: options are 1 (male present at bower) and 0 (male absent from bower)
#fitness_states: keeps track of number of matings a given male has had
#t: time
#nodes: total number of male bowerbirds in the network
#node_dist: the node by nde matrix with distances between each node
#node_graph: matrix with individuals 1 through nodes numbered by going through each row




import math
import random
import numpy as numpy
import matplotlib.pyplot as plt


# makes a ticket
def ticketgenerator(tau,t, o, a, targ):
    ticket={
        'tau': tau,
        'time': t,
        'owner': o,
        'action': a,
        'target': targ
    }
    return ticket;

#writes the edges to a network of n birds where everyone is connected to everyone else 
def networkwriter(n):
    connect=[0]*n
    for i in range(0, n): 
        a=list(range(n)) #generating a list that goes from 0 to n-1
        a.remove(i) #individual removes themselves from the network
        connect[i]=a #corresponds to the list of people person i is linked to
    return connect; #returns a list of lists with all the connections in the network


# function for determining the next time based on our rate parameters
def nexttau(action):
    switcher = {
        -1: numpy.random.normal(loc=.1583, scale=.09755, size=1)[0], #choose when to leave bower (generate a tau for bower stay)
        -2: numpy.random.gamma(shape=1.5, scale=5, size=1)[0]/60, #choose when to stop foraging (generate a tau for foraging)
        -3: numpy.random.uniform(FV_param[0], FV_param[1]), #FV_param... totally arbitrary so we should think about it
        -5: MA_param, #in the future we'll do something with it
        -7: RB_param #in the future we'll do something with it 
    }
    x=switcher.get(action)
    return x
    

#adds new tickets to timeline -- Stefano, any more efficient suggestions for keeping timeline in order?
def addtotimeline(tic, timeline):
    if not timeline:
        timeline.append(tic)
    else:
        ind=len(timeline)-1
        end=0
        while (tic['time']<timeline[ind]['time'] and end==0): #moves backwards until it finds where to place the ticket based on the listed times
            ind=ind-1 
            if(ind<0):
                end=1
        ind=ind+1
        timeline.insert(ind, tic)
    
def SBtickethandler(SB_tic, timeline, SB_param, t_max, male_states, MT_vs_FG, visit_preferences):
    ow=SB_tic['owner'] 
    if male_states[ow]==1: #because any male who is at his bower was last repairing his bower (not coming from FG or MR) 
        bower_states[ow]=1
    male_states[ow]=1
    t=SB_tic['time']
    if bower_states[ow]==0: #if male returns to a destroyed bower, must repair it 
        RB_tic=ticketgenerator(0,t,ow,-7,ow) #immediately, RB_tic starts 
        addtotimeline(RB_tic, timeline)
    else:
        MT_FG_time=nexttau(-1)+t #We use SB_param because the time represents when the bird leaves the bower
        if MT_FG_time<t_max:
            decider=random.random()
            if decider<MT_vs_FG: #if transition to MT
                targ=numpy.random.choice(list(range(nodes)), p=visit_preferences[ow]) #choosing the male to maurad based on visit preferences
                MT_tic=ticketgenerator(MT_FG_time-t,MT_FG_time,ow,-4, targ)
                addtotimeline(MT_tic, timeline)
            else:
                FG_tic=ticketgenerator(MT_FG_time-t,MT_FG_time,ow,-2, ow) #-2 denotes foraging action
                addtotimeline(FG_tic,timeline)

def FGtickethandler(FG_tic, timeline, FG_param, t_max, male_states):
    ow=FG_tic['owner'] 
    male_states[ow]=0
    t=FG_tic['time']
    SB_time=nexttau(-2)+t #We use FG_param because the time represents when the bird returns to the bower
    if SB_time<t_max:
        SB_tic=ticketgenerator(SB_time-t,SB_time,ow,-1, ow) #-1 denotes staying at bower
        addtotimeline(SB_tic,timeline)
        
def MTtickethandler(MT_tic,timeline,travel_times, t_max,male_states):
    ow=MT_tic['owner']
    male_states[ow]=0 #male is no longer at his bower
    targ=MT_tic['target']
    MA_time=t+travel_times[ow][targ] #determining the time it takes to go from the owner's bower back to the target's bower
    if MA_time<t_max:
        MA_tic=ticketgenerator(MA_time-t,MA_time, ow, -5, targ) #generate a maurading action ticket
        addtotimeline(MA_tic, timeline)
            
def MAtickethandler(MA_tic, timeline, t_max, male_states, bower_states):
    ow=MA_tic['owner'] 
    targ=MA_tic['target']
    t=MA_tic['time']
    if male_states[targ]==0 and bower_states[targ]==1: #if the bower is intact and its owner is absent
        bower_states[targ]=0
        MR_time=nexttau(action)+t #not negative, but just goes to default
    else: #in all other cases bowerbird immediately leaves
        MR_time=t    
    if MR_time<t_max:
        MR_tic=ticketgenerator(MR_time-t,MR_time, ow, -6, targ)
        addtotimeline(MR_tic, timeline)

            
def MRtickethandler(MR_tic,timeline,travel_times,t_max):
    ow=MR_tic['owner']
    targ=MR_tic['target'] #accesing the male the owner chose to maurad based on visit preferences
    SB_time=t+travel_times[ow][targ] #determining the time it takes to go from target's bower back to owner's bower
    if SB_time<t_max:
        SB_tic=ticketgenerator(SB_time-t,SB_time, ow, -1, ow) #generate stay at bower ticket
        addtotimeline(SB_tic, timeline)
            
def RBtickethandler(RB_tic, timeline, t_max):
    ow=RB_tic['owner']
    t=RB_tic['time']
    SB_time=nexttau(-4)+t
    if SB_time<t_max:
        SB_tic=ticketgenerator(SB_time-t, SB_time, ow, -1, ow)
        addtotimeline(SB_tic, timeline)
    
def FVtickethandler(FV_tic, timeline, FV_param, t_max, male_states, fitness_states, nodes, visit_preferences, travel_times, success_rate, success_times, recents_list, max_visits):
    ow=FV_tic['owner']
    t=FV_tic['time']
    r=random.random()
    if bower_states[ow]==1 and male_states[ow]==1 and r<success_rate: #if the bower is intact and the male is present
        recents_list=[]
        fitness_states[ow]=fitness_states[ow]+1 #assumption: female always mates if bower is intact and male present
        success_times.append(t)
        new_FV_ow=numpy.random.choice(list(range(nodes)))
        decider=random.random() #decide whether female will mate for a second time
        if decider<.8: #half the time she'll mate again due to predation (or for the heck of it -- PJ said 4-5 matings)
            new_FV_time=t+nexttau(-3)
            if new_FV_time<t_max:
                new_FV_tic=ticketgenerator(new_FV_time-t,new_FV_time, new_FV_ow, -3, new_FV_ow)
                addtotimeline(new_FV_tic, timeline)        
    else: #if female does not successfully mate
        recents_list.append(ow)
        if len(recents_list)==min(max_visits, nodes): 
            recents_list=[]
            print(recents_list)
            new_FV_ow=numpy.random.choice(list(range(nodes)))
            new_FV_time=t+nexttau(-3)
            if new_FV_time<t_max:
                new_FV_tic=ticketgenerator(new_FV_time-t,new_FV_time, new_FV_ow, -3, new_FV_ow)
                addtotimeline(new_FV_tic, timeline)
        else:
            new_FV_ow=-1 #just so that the code goes into the while loop the first iteration
            while(new_FV_ow in recents_list or new_FV_ow==-1):
                new_FV_ow=numpy.random.choice(list(range(nodes)), p=visit_preferences[ow]) #choose a male based on preference (a function of distance)
            new_FV_time=t+travel_times[ow][new_FV_ow] #she goes directly to this male
            if new_FV_time<t_max:
                new_FV_tic=ticketgenerator(new_FV_time-t,new_FV_time, new_FV_ow, -3, new_FV_ow)
                addtotimeline(new_FV_tic, timeline)
    return recents_list



    
    
    
#FOR LATER
#def MTtickethandler (MT_tic, timeline, )

#distance function
def travel(nodes,d,bird_speed):
    node_dist= numpy.array([[-1.0]*nodes]*nodes) #(dist in m)initialize a nodes-by-nodes matrix (1st nrows, 2nd ncols)
    sqrt_nodes=int(math.sqrt(nodes)) 
    node_graph=numpy.arange(nodes)
    node_mat=node_graph.reshape(sqrt_nodes,sqrt_nodes)
    for i in range(sqrt_nodes):
        for j in range (sqrt_nodes):
            n1=node_mat[i][j]
            for a in range(sqrt_nodes):
                for b in range(sqrt_nodes):
                    n2=node_mat[a][b]
                    if n1<n2:
                        d12=math.hypot((i-a)*d,abs(j-b)*d)
                        node_dist[n1][n2]=d12
                        node_dist[n2][n1]=d12
                    if n1==n2:
                        node_dist[n1][n2]=0
    travel_times=numpy.array([[0.0]*nodes]*nodes)
    for i in range(nodes):
        for j in range(nodes):
            travel_times[i][j]=node_dist[i][j]/bird_speed
    return [node_dist,travel_times, node_mat]




bird_speed=12*3600 #m/hr (12m/s)
nodes=100
d=150.0
travel_mats=travel(nodes,d,bird_speed)
node_dist=travel_mats[0]
travel_times=travel_mats[1]
#print(travel_mats[0].astype(int))
#print(travel_mats[1])




#solve for lambda 
improb=0.99
improb_distance=800
lamb=-math.log(1-improb)/improb_distance

#will write female preference based on cumulative exponential decay (lambda=.00576)
def female_probs(node_dist, nodes, lamb):
    visit_preferences=numpy.array([[0.]*nodes]*nodes)
    for i in range(nodes):
        for j in range(nodes):
            if i!=j:
                visit_preferences[i][j]=math.exp(-lamb*node_dist[i][j])        
        visit_preferences[i]=visit_preferences[i]/sum(visit_preferences[i])
    return visit_preferences

visit_preferences=female_probs(travel_times, nodes, lamb)
#print(visit_preferences)
#print(visit_preferences[1])


In [2]:
#just a test

#Parameters:
t=0.0 #start at time 0
t_max=12*4*30 #2 months in hrs (assume no nights so 12 hr = 1 day)
timeholder=0
timeline=[] #initialize the timeline
#We've moved away from using these params with expovariate (how we handle FV is still TBD)
#So THE BELOW 3 SHOULD BE COMMENTED OUT (but rn we don't want the handler funcs to yell at us)
FV_param=[1, t_max/2] #5days, tmax/3 #0.2 #nodes/0.19 #totally random
FG_param=3 #larger values lead to foraging ending earlier, because taus are smaller
SB_param=1.5 #larger values lead to bowerstay ending earlier, because taus are smaller
RB_param=.5 #made up -- takes 30mins (every time -- no distribution) to repair bower
MA_param=.1 #made up -- takes 6mins (every time -- no distribution) to maraud bower
MT_vs_FG=.05 #compared 20 feedings/day to .1 maraudings/hr
nodes=100
bower_states=[1]*nodes
male_states=[1]*nodes
fitness_states=[0]*nodes
success_rates=1
success_times=[]
recents_list=[]
max_visits=6 #STEEEEVEEEE
  
for i in range(int(3*nodes)):
    first_male=random.choice(list(range(nodes))) #The lucky bowerbird that is chosen by the female first
    first_female_time=t+nexttau(-3) #determine the time when the first female arrives
    first_female_tic=ticketgenerator(first_female_time-t,first_female_time, first_male, -3, first_male) #first female ticket
    addtotimeline(first_female_tic, timeline)

for i in range(nodes):
    SB_tic=ticketgenerator(t, t, i, -1, i)
    addtotimeline(SB_tic,timeline)
    #init_FG_time=t+nexttau(-1) #Init FG time indicates when SB ends, thus the tau is generated by SB_param
    #init_FG_tic=ticketgenerator(init_FG_time-t,init_FG_time, i, -2, i)
    #addtotimeline(init_FG_tic, timeline)
        


In [3]:
#while loop

while t<t_max:
    #print(t)
    if timeholder>(len(timeline)-1): #if timeholder exceeds max timeline index
        #print(sum(ppl_state)/num_ppl)
        print('end')
        break
    next_tic=timeline[timeholder] #look at the ticket corresponding to where we are on the timeline
    action=next_tic['action']
    switcher = {
        -1: SBtickethandler(next_tic, timeline, SB_param, t_max, male_states, MT_vs_FG, visit_preferences),
        -2: FGtickethandler(next_tic,timeline,FG_param,t_max,male_states),
        -3: FVtickethandler(next_tic, timeline, FV_param, t_max, male_states, fitness_states, nodes, visit_preferences, travel_times, success_rates, success_times, recents_list, max_visits),
        -4: MTtickethandler(next_tic,timeline,travel_times, t_max,male_states),
        -5: MAtickethandler(next_tic, timeline, t_max, male_states, bower_states),
        -6: MRtickethandler(next_tic,timeline,travel_times,t_max),
        -7: RBtickethandler(next_tic, timeline, t_max) 
    }
    recents_list=switcher.get(action)
    #if (next_tic['action']==-1): #if it's a stay-at-bower ticket, use the corresponding handler
    #    SBtickethandler(next_tic,timeline,SB_param,t_max,male_states)
    #elif (next_tic['action']==-2): #if it's a forage ticket use the corresponding handler
    #    FGtickethandler(next_tic,timeline,FG_param,t_max,male_states)
    #else: #if it's a female visit ticket, use the corresponding handler
    #    print("passing to FVtickethandler")
    #    recents_list=FVtickethandler(next_tic, timeline, FV_param, t_max, male_states, fitness_states, nodes, female_preferences, travel_times, success_rate, success_times, recents_list, max_visits)
    t=next_tic['time'] #new time based on the ticket we just read
    timeholder=timeholder+1 #increment timeholder

print(timeline)


TypeError: unsupported operand type(s) for +: 'NoneType' and 'float'

In [ ]:
d={}


for i in range(nodes):
    d["tl{0}".format(i)]=[]
for j in range(len(timeline)):
    #for i in range(nodes):
     #   if timeline[j]['owner']==i:
    d["tl{0}".format(timeline[j]['owner'])].append(timeline[j])
    

print("tl0----")
print(d["tl0"])
print("tl1----")
print(d["tl1"])
print("tl2----")
print(d["tl2"])
print("tl3----")
print(d["tl3"])  

for i in range(nodes):
    tl=d["tl{0}".format(i)]
    l=len(tl)
    ta=[0]
    fem=[]
    for j in range(l):
        if tl[j]['action']!=-3: #female visit taus don't mean anything for the male bc she isn't necessarily coming back to the same male
            ta.append(tl[j]['time'])
        else:
            fem.append(tl[j]['time'])
    d["times{0}".format(i)]=ta
    d["fvtimes{0}".format(i)]=fem

#print("times0----")
#print(d["times0"])
#print("times1----")
#print(d["times1"])
#print("times2----")
#print(d["times2"])
#print("times3----")
#print(d["times3"]) 

#append tmax - sumoftaus to each list (so it reaches tmax exactly)
for i in range(nodes):
    d["times{0}".format(i)].append(t_max)


import csv
timess=[d["times0"],d["times1"],d["times2"],d["times3"]]
csvfile = "example_times2"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(timess)
femtimess=[]
for i in range(nodes):
    femtimess.append(d["fvtimes{0}".format(i)])

csvfile = "example_fvtimes"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(femtimess)

           
    

In [ ]:
for i in range(nodes):
    tl=d["tl{0}".format(i)]
    l=len(tl)
    ta=[]
    accSB=0
    accFG=0
    for j in range(l):
        if tl[j]['action']!=-3: #female visit taus don't mean anything for the male bc she isn't necessarily coming back to the same male
            ta.append(tl[j]['tau'])
            if tl[j]['action']==-1:
                accSB=accSB+tl[j]['tau']
            else:
                accFG=accFG+tl[j]['tau']
    
    d["taus{0}".format(i)]=ta
    print("Bird {:d}'s % time at bower is {:f}".format(i, accFG/t_max)) ##VERY ODD -- could it be that FG/SB are switched

#print("taus0----")
#print(d["taus0"])
#print(sum(d["taus0"]))
#print("taus1----")
#print(d["taus1"])
#print(sum(d["taus1"]))
#print("taus2----")
#print(d["taus2"])
#print(sum(d["taus2"]))
#print("taus3----")
#print(d["taus3"]) 
#print(sum(d["taus3"]))


#append tmax - sumoftaus to each list (so it reaches tmax exactly)
#for i in range(nodes):
#    d["taus{0}".format(i)].append(t_max-sum(d["taus{0}".format(i)]))

    
#print(sum(d["taus0"]))
#print(sum(d["taus1"]))
#print(sum(d["taus2"]))
#print(sum(d["taus3"]))  

In [ ]:
#Create travel_time__linear_p (might use later)
travel_times_linear_p=numpy.array([[0.0]*nodes]*nodes)
for i in range(nodes):
    for j in range(nodes):
        if i==j: #if the row and the column index are the equal
            travel_times[i][j]=0 #There is 0 probability they will travel to their own bower when choosing to fly to another bower
        else: 
            if i==0: #if on the first row of the probability matrix
                travel_times_linear_p[i][j]=travel_times[i][1]/travel_times[i][j] #skip the first element in the row (because it equals 0) and compare probabilities of other elements in the row based on travel distance (negative linear relationship btwn distance and prob)
            else:
                travel_times_linear_p[i][j]=travel_times[i][0]/travel_times[i][j] #use the first element in a row and compare probabilities of other elements in the row based on distance (negative linear relationship btwn distance and prob)
    travel_times_linear_p[i]=travel_times_linear_p[i]/sum(travel_times_linear_p[i]) #normalize the probabilities so they add up to 1
print(travel_times_linear_p)



In [ ]:
nodes=100
print(fitness_states)
print(sum(fitness_states))
j=numpy.mean(fitness_states)
print(j)
l=numpy.var(fitness_states)
print(l)
f_s=numpy.array(fitness_states)
#was testing if edges are at a disadvantage
x=[i for i, e in enumerate(fitness_states) if e == 0]
print(x)
sqrt_nodes=int(math.sqrt(nodes))
f_s2=f_s.reshape(sqrt_nodes,sqrt_nodes)
print(f_s2)
print((nodes-(4*(sqrt_nodes - 1)))/nodes)
exp=[[2.64]*sqrt_nodes]*sqrt_nodes
print(exp)
print(f_s2)



ra=numpy.random.normal(loc=j, scale=numpy.sqrt(l), size=nodes)
ra2=ra.reshape(sqrt_nodes,sqrt_nodes)

diff1=numpy.subtract(exp, f_s2)
diff2=numpy.subtract(exp, ra2)
#print(diff1)
#print(diff2)
print(sum(abs(diff1.reshape(100,1)))/100)
print(sum(abs(diff2.reshape(100,1)))/100)
#FOUND NO SIG DIFFERENCE -- var

In [ ]:
#for now, we write a simple example network with 4 bowers forming a square
nodes=4
d=150
distances= numpy.array([[d]*nodes]*nodes) #(dist in m)initialize a nodes-by-nodes matrix (1st nrows, 2nd ncols)

for i in range(nodes):
    distances[i][i]=9999.0 #or math.inf but that makes the matrix disp ugly
for i in range(2):
    distances[i][i+2]=((2*(d**2)))**(1/2)
    distances[i+2][i]=((2*(d**2)))**(1/2)
print(distances)
    
#HOW TO REPRESENT INDIVIDUAL'S DIST FROM SELF? nan?
bird_speed=12*3600 #m/hr (12m/s)
travel_times=numpy.array([[0.0]*nodes]*nodes)
for i in range(nodes):
    for j in range(nodes):
        travel_times[i][j]=distances[i][j]/bird_speed
print(travel_times) #right now the numbers on the diagonals aren't accurate, but we can't have the bird choosing to go to it's own node



In [ ]:
print(success_times)

In [ ]:
plt.hist(success_times)
plt.show()

In [ ]:
print(len(femtimess))
#print(femtimess)
fts = [item for sublist in femtimess for item in sublist]
print(len(fts))

In [ ]:
# function for determining the next time based on our rate parameters
def nexttau(action):
    switcher = {
        -1: numpy.random.normal(loc=.1583, scale=.09755, size=1)[0], #choose when to leave bower (generate a tau for bower stay)
        -2: numpy.random.gamma(shape=1.5, scale=5, size=1)[0]/60, #choose when to stop foraging (generate a tau for foraging)
        -3: numpy.random.uniform(FV_param[0], FV_param[1]), #FV_param... totally arbitrary so we should think about it
        -5: MA_param, #in the future we'll do something with it
        -7: RB_param #in the future we'll do something with it 
    }
    x=switcher.get(action)
    return x
x=[-1,-2,-3,-5,-7]
for i in x:
    print(nexttau(i)+t)
print(t)

In [4]:
print(timeline)

[{'tau': 0.0, 'time': 0.0, 'owner': 0, 'action': -1, 'target': 0}, {'tau': 0.0, 'time': 0.0, 'owner': 1, 'action': -1, 'target': 1}, {'tau': 0.0, 'time': 0.0, 'owner': 2, 'action': -1, 'target': 2}, {'tau': 0.0, 'time': 0.0, 'owner': 3, 'action': -1, 'target': 3}, {'tau': 0.0, 'time': 0.0, 'owner': 4, 'action': -1, 'target': 4}, {'tau': 0.0, 'time': 0.0, 'owner': 5, 'action': -1, 'target': 5}, {'tau': 0.0, 'time': 0.0, 'owner': 6, 'action': -1, 'target': 6}, {'tau': 0.0, 'time': 0.0, 'owner': 7, 'action': -1, 'target': 7}, {'tau': 0.0, 'time': 0.0, 'owner': 8, 'action': -1, 'target': 8}, {'tau': 0.0, 'time': 0.0, 'owner': 9, 'action': -1, 'target': 9}, {'tau': 0.0, 'time': 0.0, 'owner': 10, 'action': -1, 'target': 10}, {'tau': 0.0, 'time': 0.0, 'owner': 11, 'action': -1, 'target': 11}, {'tau': 0.0, 'time': 0.0, 'owner': 12, 'action': -1, 'target': 12}, {'tau': 0.0, 'time': 0.0, 'owner': 13, 'action': -1, 'target': 13}, {'tau': 0.0, 'time': 0.0, 'owner': 14, 'action': -1, 'target': 14},